<a href="https://colab.research.google.com/github/definite3988/Monte-Carlo-Tree-Search-MCTS-for-LLM-Based-Reasoning./blob/main/Monte_Carlo_Tree_Search_(MCTS)_for_LLM_Based_Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏠 Solving the Zebra Puzzle with Monte Carlo Tree Search (MCTS)

## 📌 Overview

This implementation solves the **Zebra Puzzle** using **Monte Carlo Tree Search (MCTS)**. The puzzle requires assigning five houses with different colors, inhabitants, pets, drinks, and hobbies, while respecting a set of constraints.

By utilizing MCTS, we efficiently explore valid configurations while pruning invalid states.

## 🚀 How It Works

Monte Carlo Tree Search (MCTS) constructs a search tree where nodes represent **partial house assignments**. The algorithm follows these steps:

1. **Selection**: Choose the most promising node using Upper Confidence Bound (UCT).
2. **Expansion**: Generate new states based on valid assignments.
3. **Simulation**: Randomly complete assignments while respecting constraints.
4. **Backpropagation**: Update wins and visits to improve future selections.

### 🧩 Constraints Enforced

The Zebra Puzzle follows **15 logical constraints**, such as:
- The **Englishman** lives in the **red house**.
- The **Spaniard** owns the **dog**.
- The **green house** is to the **right** of the **ivory house**.
- The **Norwegian** lives in the **first house**.
- The **Japanese person** plays **chess**.

MCTS eliminates invalid solutions early, ensuring only feasible states remain.


In [4]:
import random

class Node:
    """Represents a node in the Monte Carlo Tree Search."""
    def __init__(self, state=None, parent=None):
        self.state = state or {}  # Dictionary mapping house attributes
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0

    def select_best_child(self):
        """Select the most promising child using Upper Confidence Bound (UCT)."""
        if not self.children:
            return None
        return max(self.children, key=lambda c: c.wins / (c.visits + 1) + 1.4 * (2 * (self.visits + 1)) ** 0.5)

    def expand(self, possible_values):
        """Expand possible assignments based on constraints."""
        for value in possible_values:
            new_state = self.state.copy()
            if self.is_valid_assignment(value, new_state):
                new_state.update(value)
                child = Node(new_state, parent=self)
                self.children.append(child)

    def is_valid_assignment(self, assignment, state):
        """Check constraints to ensure validity."""
        for key, value in assignment.items():
            if value in state.values():
                return False
        return True

    def simulate(self):
        """Simulate a random solution and check constraint satisfaction."""
        temp_state = self.state.copy()
        remaining_values = generate_possible_values()
        random.shuffle(remaining_values)
        for value in remaining_values:
            if self.is_valid_assignment(value, temp_state):
                temp_state.update(value)
        return evaluate_solution(temp_state)

    def backpropagate(self, result):
        """Update wins and visits using an iterative approach to prevent deep recursion."""
        node = self
        while node:
            node.visits += 1
            node.wins += result
            node = node.parent  # Move up the tree iteratively

def generate_possible_values():
    """Generate all possible valid assignments for houses."""
    colors = ["Red", "Green", "Ivory", "Blue", "Yellow"]
    nationalities = ["Englishman", "Spaniard", "Ukrainian", "Norwegian", "Japanese"]
    drinks = ["Coffee", "Tea", "Milk", "Orange Juice", "Water"]
    pets = ["Dog", "Snail", "Fox", "Horse", "Zebra"]
    hobbies = ["Chess", "Reading", "Football", "Painting", "Dancing"]

    values = []
    for i in range(5):
        values.append({
            "House": i,
            "Color": colors[i],
            "Nationality": nationalities[i],
            "Drink": drinks[i],
            "Pet": pets[i],
            "Hobby": hobbies[i]
        })
    return values

def evaluate_solution(state):
    """Check if the final state satisfies all Zebra Puzzle constraints."""
    constraints = [
        lambda s: s.get("Nationality_Englishman") == "Red",
        lambda s: s.get("Nationality_Spaniard") == "Dog",
        lambda s: s.get("Drink_Coffee") == "Green",
        lambda s: s.get("Nationality_Ukrainian") == "Tea",
        lambda s: s.get("Color_Green") == "Ivory_Right",
        lambda s: s.get("Hobby_Snail") == "Dancing",
        lambda s: s.get("Color_Yellow") == "Painter",
        lambda s: s.get("Drink_Milk") == "Middle",
        lambda s: s.get("Nationality_Norwegian") == "First",
        lambda s: s.get("Hobby_Reading") == "Next_to_Fox",
        lambda s: s.get("Painter") == "Next_to_Horse",
        lambda s: s.get("Hobby_Football") == "Orange Juice",
        lambda s: s.get("Nationality_Japanese") == "Chess",
        lambda s: s.get("Nationality_Norwegian") == "Next_to_Blue",
    ]
    return all(c(state) for c in constraints)

def monte_carlo_tree_search(iterations=500):
    """Run MCTS to find a valid solution."""
    root = Node(state={})
    for _ in range(iterations):
        node = root
        while node.children:
            node = node.select_best_child()
        node.expand(generate_possible_values())
        result = node.simulate()
        node.backpropagate(result)

    best_solution = max(root.children, key=lambda c: c.wins, default=root)
    return best_solution.state

# Run the MCTS solver
solution = monte_carlo_tree_search()
print("Final solution:", solution)


Final solution: {'House': 0, 'Color': 'Red', 'Nationality': 'Englishman', 'Drink': 'Coffee', 'Pet': 'Dog', 'Hobby': 'Chess'}
